In [1]:
# --- CELL: Importing Required Libraries ---
# This cell imports all necessary libraries for building a Multiclass CNN Image Classifier.
# - TensorFlow (tf): Core deep learning framework.
# - Sequential: Used to build the CNN model by stacking layers linearly.
# - ImageDataGenerator: For loading images from directories and applying real-time data augmentation.
# - Conv2D: 2D Convolutional layer for extracting spatial features from images.
# - MaxPooling2D: Reduces spatial dimensions by taking max value in each pooling window.
# - Flatten: Converts 3D feature maps into a 1D vector for Dense layers.
# - Dense: Fully connected layer for classification.
# - Dropout: Regularization technique to prevent overfitting by randomly dropping neurons.
# - BatchNormalization: Normalizes activations between layers for stable and faster training.
# - Input: Defines the input shape of the model.

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization,Input

# OUTPUT: A SciPy version warning may appear - this is a compatibility notice between NumPy and SciPy
# and does NOT affect the code. It can be safely ignored.

C:\Users\elwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.4.1)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [2]:
# --- CELL: Setting Up Image Data Generators (Train & Validation) ---
# This cell configures data loading and augmentation for multiclass classification (3 classes).
#
# Parameters:
#   - img_size=(128,128): All images resized to 128x128 pixels.
#   - batch_size=32: Images are processed in batches of 32.
#
# TRAINING DATA AUGMENTATION:
#   - rescale=1./255: Normalizes pixel values from [0,255] to [0,1].
#   - rotation_range=20: Random rotation up to 20 degrees.
#   - zoom_range=0.2: Random zoom in/out by 20%.
#   - width/height_shift_range=0.1: Random horizontal/vertical shifts by 10%.
#   - horizontal_flip=True: Random horizontal flipping.
#   - validation_split=0.2: Reserves 20% of training data for validation.
#
# class_mode='categorical': One-hot encoded labels for multiclass classification (3 classes).

img_size=(128,128)
batch_size=32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_genrator= train_datagen.flow_from_directory(
    r"C:\Users\elwin\OneDrive\Desktop\Multiclass cnn , car, plane\train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_genrator=train_datagen.flow_from_directory(
    r"C:\Users\elwin\OneDrive\Desktop\Multiclass cnn , car, plane\test",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# OUTPUT:
# - Found 2400 training images belonging to 3 classes (e.g., airplanes, cars, ships).
# - Found 115 validation images belonging to 3 classes.
# The 3 classes are loaded from subdirectory names in the dataset folder.

Found 2400 images belonging to 3 classes.
Found 115 images belonging to 3 classes.


In [3]:
# --- CELL: Defining the CNN Model Architecture for Multiclass Classification ---
# This cell builds a Sequential CNN with 3 convolutional blocks and a classification head.
#
# Architecture breakdown:
# BLOCK 1: Conv2D(32 filters, 3x3, ReLU) -> BatchNormalization -> MaxPooling2D(2x2)
#   - Extracts low-level features (edges, textures) from 128x128x3 input images.
# BLOCK 2: Conv2D(64 filters) -> BatchNormalization -> MaxPooling2D
#   - Captures mid-level patterns (shapes, parts of objects).
# BLOCK 3: Conv2D(128 filters) -> BatchNormalization -> MaxPooling2D
#   - Learns high-level, class-specific features.
# CLASSIFIER:
#   - Flatten: Converts 3D feature maps to 1D vector (14x14x128 = 25,088 features).
#   - Dense(128, ReLU): Fully connected layer for learning feature combinations.
#   - Dropout(0.5): Drops 50% of neurons during training to prevent overfitting.
#   - Dense(3, softmax): Output layer with 3 neurons (one per class) and softmax activation
#     for multi-class probability distribution.

model=Sequential([
    Input(shape=(128,128,3)),
    
    Conv2D(32,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128,(3,3),activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(3,activation='softmax')
    
])

In [4]:
# --- CELL: Compiling the Model and Displaying Architecture Summary ---
# Compilation configures the model for training:
#   - optimizer='adam': Adam optimizer with adaptive learning rate.
#   - loss='categorical_crossentropy': Appropriate loss function for multi-class classification
#     with one-hot encoded labels.
#   - metrics=['accuracy']: Tracks accuracy during training.

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# OUTPUT (model.summary):
# The model has 3 convolutional blocks (Conv2D + BatchNorm + MaxPool) and a Dense classifier.
# Key observations:
# - Conv layers increase filters: 32 -> 64 -> 128 (progressively deeper feature extraction).
# - Spatial dimensions reduce: 126->63->61->30->28->14 through convolution and pooling.
# - After Flatten: 25,088 features (14x14x128).
# - Dense(128) layer has the most params: 3,211,392 (due to large flattened input).
# - Output Dense(3) with softmax gives probability for each of the 3 classes.
# - Total params: 3,305,923 (12.61 MB) | Trainable: 3,305,475 | Non-trainable: 448 (BatchNorm).

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,923 (12.61 MB)

 Trainable params: 3,305,475 (12.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [5]:
# --- CELL: Training the CNN Model ---
# This cell trains the model on the vehicle/aircraft image dataset for 16 epochs.
#   - train_genrator: Feeds augmented training images.
#   - epochs=16: The model sees the full training data 16 times.
#   - validation_data=val_genrator: Validates after each epoch to monitor generalization.
# Training history is stored for plotting accuracy/loss curves later.

history=model.fit(train_genrator,epochs=16,validation_data=val_genrator)

# OUTPUT (Training Logs):
# The model trained for 16 epochs (75 batches/epoch = 2400 images / 32 batch size).
# Key observations:
# - Epoch 1:  Train Acc = 64.92%, Val Acc = 33.04% (random-level performance initially)
# - Epoch 7:  Train Acc = 84.83%, Val Acc = 78.26% (significant improvement)
# - Epoch 14: Train Acc = 89.33%, Val Acc = 93.91% (best val accuracy)
# - Epoch 16: Train Acc = 90.50%, Val Acc = 88.70% (final epoch)
# - Validation loss shows high variance (fluctuating), suggesting the model struggles slightly
#   with generalization on the small validation set (115 images).
# - Training accuracy steadily improves from ~65% to ~90.5%.
# - The model achieves reasonable performance but could benefit from more data or fine-tuning.

Epoch 1/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 70s 904ms/step - accuracy: 0.6492 - loss: 2.1842 - val_accuracy: 0.3304 - val_loss: 9.9661
Epoch 2/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 619ms/step - accuracy: 0.7333 - loss: 0.7120 - val_accuracy: 0.4435 - val_loss: 15.3270
Epoch 3/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 620ms/step - accuracy: 0.7987 - loss: 0.5919 - val_accuracy: 0.5217 - val_loss: 12.5937
Epoch 4/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 46s 607ms/step - accuracy: 0.8325 - loss: 0.5161 - val_accuracy: 0.5739 - val_loss: 4.6275
Epoch 5/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 623ms/step - accuracy: 0.8292 - loss: 0.5142 - val_accuracy: 0.4087 - val_loss: 4.4718
Epoch 6/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 620ms/step - accuracy: 0.8521 - loss: 0.4588 - val_accuracy: 0.4870 - val_loss: 2.1110
Epoch 7/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 46s 607ms/step - accuracy: 0.8483 - loss: 0.4435 - val_accuracy: 0.7826 - val_loss: 0.8778
Epoch 8/16
75/75 ━━━━━━━━━━━━━━━━━━━━ 47s 618ms/step - accuracy: 0.8567 - loss: 0.4052 - val_ac

In [6]:
# --- CELL: Evaluating the Model on Validation/Test Data ---
# This cell evaluates the trained model on the validation set to get the final test accuracy and loss.
# model.evaluate() runs a forward pass on all validation images without updating weights.

test_loss, test_acc = model.evaluate(val_genrator)
print("Test Accuracy:", test_acc)

# OUTPUT:
# - Test Accuracy: 85.22% (0.852173924446106)
# - Test Loss: 0.3662
# The model correctly classifies ~85% of the validation images across 3 classes.
# This is a decent result given the relatively small dataset (2400 train, 115 val images).

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - accuracy: 0.8522 - loss: 0.3662
Test Accuracy: 0.852173924446106


In [7]:
# --- CELL: Making a Single Image Prediction ---
# This cell loads a single test image (an aeroplane), preprocesses it, and predicts its class.
#
# Preprocessing steps:
#   1. Load image and resize to 128x128 (matching training input size).
#   2. Convert to numpy array using img_to_array.
#   3. Normalize pixel values to [0,1] by dividing by 255.
#   4. Expand dimensions: (128,128,3) -> (1,128,128,3) to create a batch of 1.
#   5. model.predict() returns probability for each of the 3 classes.
#   6. np.argmax() gets the index of the highest probability class.
#   7. Maps the index to the class label using train_genrator.class_indices.

import numpy as np
from tensorflow.keras.preprocessing import image

img=image.load_img(r"C:\Users\elwin\Downloads\aeroplane image.jpeg",target_size=(128,128))
img_array=image.img_to_array(img)
img_array=img_array/255
img_array=np.expand_dims(img_array,axis=0)

prediction=model.predict(img_array)
predicted_class=np.argmax(prediction)

labels=list(train_genrator.class_indices.keys())
print('predicted :',labels[predicted_class])

# OUTPUT:
# predicted : airplanes
# The model correctly identified the uploaded image as an 'airplane' class.
# This confirms the model is working as expected for inference.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
predicted : airplanes


In [8]:
# --- CELL: Importing Gradio and PIL for Web Interface ---
# - Gradio (gr): Library for creating interactive ML web UIs.
# - PIL (Image): For image format conversion during preprocessing.

import gradio as gr
from PIL import Image

In [9]:
# --- CELL: Setting Up the Gradio Web Interface for Vehicle Classification ---
# This cell creates a Gradio-based web app for real-time vehicle/aircraft classification.
#
# Prediction function (vechicle_detection):
#   1. Converts uploaded numpy image to PIL Image.
#   2. Resizes to 128x128 (matching training size).
#   3. Normalizes pixel values to [0,1].
#   4. Expands dimensions to add batch axis.
#   5. Runs prediction through the model.
#   6. np.argmax() gets the class with highest probability.
#   7. Maps index to class label (e.g., 'airplanes', 'cars', 'ships').
#
# Gradio Interface:
#   - Input: Image upload widget.
#   - Output: Textbox showing predicted vehicle class.

models=model

# image preprocessing funtion
def vechicle_detection(image):
    image=Image.fromarray(image)
    image=image.resize((128,128))
    image=np.array(image)/255.0
    image=np.expand_dims(image,axis=0)

    predictions=models.predict(image)
    predict_class=np.argmax(predictions)
    label=list(train_genrator.class_indices.keys())
    return label[predict_class]
        

interface=gr.Interface(
    fn=vechicle_detection,
    inputs=gr.Image(type='numpy',label='Upload Image'),
    outputs=gr.Textbox(label='Prediction'),
    title='Vechicle image Classifier',
    description='upload an image to identify the animal'
)

In [10]:
# --- CELL: Launching the Gradio Web Application ---
# This launches the Gradio interface as a local web server for the vehicle classifier.

interface.launch()

# OUTPUT: The Gradio app runs on http://127.0.0.1:7860
# An interactive iframe is embedded in the notebook.
# When images are uploaded, the model predicts the vehicle type (airplane/car/ship).
# Prediction runs in ~50ms per image (1/1 batch).
# To share publicly, use interface.launch(share=True).

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
